In [ ]:
#interactive widgets in vs code
%matplotlib inline

#dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import bagpy
from bagpy import bagreader

In [ ]:
# Read rosbag data
bag = bagreader("data/test2.bag")

# extract data from bag, and convert to pandas dataframe
accel_df = pd.read_csv(bag.message_by_topic('/imu/acceleration'))
gyro_df = pd.read_csv(bag.message_by_topic('/imu/angular_velocity'))
bag.topic_table

In [ ]:
# calculate time difference between measurements
timesteps = (accel_df['header.stamp.secs'].diff(1) + accel_df['header.stamp.nsecs'].diff(1) / 1e9).to_numpy()

# take rolling average of IMU data and store to numpy array, this is to filter noise
averaging_window = 20
vec_accel_raw = accel_df[['vector.x', 'vector.y', 'vector.z']].rolling(averaging_window).mean().to_numpy()
vec_gyro_raw = gyro_df[['vector.x', 'vector.y', 'vector.z']].rolling(averaging_window).mean().to_numpy()

# longer averaging to get bias over time
accel_bias = -accel_df[['vector.x', 'vector.y', 'vector.z']].rolling(10000).mean().to_numpy()

# replace nan values using closest value
def replace_nan(a):
    ind = np.where(~np.isnan(a))[0]
    first, last = ind[0], ind[-1]
    a[:first] = a[first]
    a[last + 1:] = a[last]
    return a

timesteps = replace_nan(timesteps)
vec_gyro_raw = replace_nan(vec_gyro_raw)


In [ ]:
# integrate angular velocity to find rotation (multiply with timestep, then cumulative sum)
vec_rotation = np.cumsum(vec_gyro_raw * timesteps[:, np.newaxis], axis=0)

# rotation calculations
vec_velocity_rotated = np.zeros_like(vec_gyro_raw)

for nr, rotation_measure in enumerate(vec_rotation):    # loop over vectors
    roll, pitch, yaw = rotation_measure
    rollMatrix = np.matrix([                    # rotation about x
        [1, 0,             0           ],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll),  np.cos(roll)]])
    pitchMatrix = np.matrix([                   # rotation about y
        [ np.cos(pitch), 0, np.sin(pitch)],
        [0,              1,  0           ],
        [-np.sin(pitch), 0, np.cos(pitch)]])
    yawMatrix = np.matrix([                     # rotation about z
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw),  np.cos(yaw), 0],
        [0,            0,           1]])

    R = yawMatrix * pitchMatrix * rollMatrix        # combined rotation matrix
    accel_rotated = np.matmul(R, [1, 0, 0])     # matrix multiplication with accel vector

    vec_velocity_rotated[nr, :] = accel_rotated        # add to array

# integrate accel and velocity to get position (multiply with timestep, then cumulative sum)
vec_position = np.cumsum(vec_velocity_rotated * timesteps[:, np.newaxis], axis=0)

In [ ]:
# calculate magnitude of drift
start_pos = vec_position[:500].mean(axis=0)
end_pos = vec_position[-500:].mean(axis=0)

drift = end_pos - start_pos
drift

In [ ]:
# plot trajectory
fig = plt.figure()
#ax = plt.axes(projection='3d')
ax = plt.axes()
xdata = vec_position[:, 0]
ydata = vec_position[:, 1]
zdata = vec_position[:, 2]

#ax.plot(*start_pos, 'gx')
#ax.plot(*end_pos, 'rx')
ax.plot(xdata, ydata)
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
#ax.set_zlabel('z [m]')
plt.show()